In [24]:
!pip install evalml

In [2]:
from urllib.request import urlopen
import pandas as pd
input_data = urlopen('https://featurelabs-static.s3.amazonaws.com/spam_text_messages_modified.csv')
data = pd.read_csv(input_data)
data.head()

,Category,Message
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,spam,FreeMsg Hey there darling it's been 3 week's n...
2,spam,WINNER!! As a valued network customer you have...
3,spam,Had your mobile 11 months or more? U R entitle...
4,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [4]:
### Independent And Dependent Features
X=data.drop('Category',axis=1)
y=data['Category']

In [5]:
y.value_counts(normalize=True)

ham     0.750084
spam    0.249916
Name: Category, dtype: float64

In [7]:
import evalml

In [8]:
#### TRain A\nd test data split
X_train,X_test,y_train,y_test=evalml.preprocessing.split_data(X,y,problem_type='binary')

In [10]:
#different type of models avaliable into eval
evalml.problem_types.ProblemTypes.all_problem_types

[<ProblemTypes.BINARY: 'binary'>,
 <ProblemTypes.MULTICLASS: 'multiclass'>,
 <ProblemTypes.REGRESSION: 'regression'>,
 <ProblemTypes.TIME_SERIES_REGRESSION: 'time series regression'>,
 <ProblemTypes.TIME_SERIES_BINARY: 'time series binary'>,
 <ProblemTypes.TIME_SERIES_MULTICLASS: 'time series multiclass'>]

In [11]:
X_train.head()

,Message
562,"Haha I heard that, text me when you're around"
1253,I'm thinking that chennai forgot to come for a...
1816,Can you tell Shola to please go to college of ...
2054,K k pa Had your lunch aha.
511,staff.science.nus.edu.sg/~phyhcmk/teaching/pc1323


In [12]:
from evalml import AutoMLSearch

In [13]:
automl=AutoMLSearch(X_train=X_train,y_train=y_train,problem_type='binary',max_batches=1,optimize_thresholds=True)

Generating pipelines to search over...
8 pipelines ready for search.


In [14]:
automl.search()


*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: catboost, extra_trees, linear_model, lightgbm, random_forest, decision_tree, xgboost



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 8.638

*****************************
* Evaluating Batch Number 1 *
*****************************

Elastic Net Classifier w/ Text Featurization Component + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.243
	High coefficient of variation (cv >= 0.2) within cross validation scores.
	Elastic Net Classifier w/ Text Featurization Component + Standard Scaler may not perform as estimated on unseen data.
Decision Tree Classifier w/ Text Featurization Component:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.802
	High coefficient of variation (cv >= 0.2) within cross validation scores.
	Decision Tree Classifier w/ Text Featurization Component may not perform as estimated on unseen data.
Random Forest Classifier w/ Tex

In [15]:
automl.rankings

,id,pipeline_name,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,3,Random Forest Classifier w/ Text Featurization...,0.154849,0.050536,0.110302,98.207418,True,{'Random Forest Classifier': {'n_estimators': ...
1,6,XGBoost Classifier w/ Text Featurization Compo...,0.178639,0.063117,0.113254,97.932010,True,"{'XGBoost Classifier': {'eta': 0.1, 'max_depth..."
2,5,Logistic Regression Classifier w/ Text Featuri...,0.214011,0.051316,0.165624,97.522538,True,{'Logistic Regression Classifier': {'penalty':...
3,4,LightGBM Classifier w/ Text Featurization Comp...,0.214580,0.072715,0.136260,97.515944,True,{'LightGBM Classifier': {'boosting_type': 'gbd...
4,1,Elastic Net Classifier w/ Text Featurization C...,0.243362,0.075438,0.157467,97.182755,True,"{'Elastic Net Classifier': {'alpha': 0.0001, '..."
5,7,Extra Trees Classifier w/ Text Featurization C...,0.252206,0.059094,0.216198,97.080377,True,{'Extra Trees Classifier': {'n_estimators': 10...
6,8,CatBoost Classifier w/ Text Featurization Comp...,0.526403,0.015739,0.512717,93.906174,False,"{'CatBoost Classifier': {'n_estimators': 10, '..."
7,2,Decision Tree Classifier w/ Text Featurization...,0.801766,0.213655,0.555179,90.718481,True,{'Decision Tree Classifier': {'criterion': 'gi...
8,0,Mode Baseline Binary Classification Pipeline,8.638305,0.025020,8.623860,0.000000,False,{'Baseline Classifier': {'strategy': 'mode'}}


In [16]:
automl.best_pipeline

pipeline = BinaryClassificationPipeline(component_graph=[TextFeaturizer, RandomForestClassifier], parameters={'Random Forest Classifier':{'n_estimators': 100, 'max_depth': 6, 'n_jobs': -1}}, custom_hyperparameters={'None':{'sampling_ratio': 0.25}}, random_seed=0)

In [17]:
best_pipeline = automl.best_pipeline

In [18]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])


************************************************************
* Random Forest Classifier w/ Text Featurization Component *
************************************************************

Problem Type: binary
Model Family: Random Forest

Pipeline Steps
1. Text Featurization Component
2. Random Forest Classifier
	 * n_estimators : 100
	 * max_depth : 6
	 * n_jobs : -1

Training
Training for binary problems.
Total training time (including CV): 23.3 seconds

Cross Validation
----------------
             Log Loss Binary  MCC Binary   AUC  Precision    F1  Balanced Accuracy Binary  Accuracy Binary  Sensitivity at Low Alert Rates # Training # Validation
0                      0.110       0.895 0.987      0.938 0.921                     0.942            0.961                           0.246      1,594          797
1                      0.144       0.854 0.980      0.919 0.888                     0.917            0.946                           0.246      1,594          797
2                   

In [19]:
### Evaluate on the test data

In [20]:
scores = best_pipeline.score(X_test, y_test,  objectives=evalml.objectives.get_core_objectives('binary'))
print(f'Accuracy Binary: {scores["Accuracy Binary"]}')

Accuracy Binary: 0.9732441471571907
